In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install Mahotas (required for LBP features)
!pip install mahotas

# Step 3: Import Necessary Libraries
import cv2
import numpy as np
import joblib
from skimage.feature import hog
import mahotas

# Step 4: Define Functions

# Preprocessing Function
def preprocess_image(image):
    # Histogram Equalization
    hist_eq = cv2.equalizeHist(image)
    # Image Sharpening
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(hist_eq, -1, kernel)
    return sharpened

# Feature Extraction Functions
def extract_sift_features(images, max_features=128):
    sift = cv2.SIFT_create()
    features = []
    for img in images:
        keypoints, descriptors = sift.detectAndCompute(img, None)
        if descriptors is None:
            descriptors = np.zeros((1, max_features))
        flattened = descriptors.flatten()
        if len(flattened) < max_features:
            flattened = np.pad(flattened, (0, max_features - len(flattened)), mode='constant')
        else:
            flattened = flattened[:max_features]
        features.append(flattened)
    return np.array(features)

def extract_hog_features(images):
    features = []
    for img in images:
        hog_features = hog(img, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        features.append(hog_features)
    return np.array(features)

def extract_lbp_features(images):
    features = []
    for img in images:
        lbp = mahotas.features.lbp(img, radius=1, points=8, ignore_zeros=False)
        features.append(lbp)
    return np.array(features)

# Combine Features
def combine_features(images):
    sift_features = extract_sift_features(images)
    hog_features = extract_hog_features(images)
    lbp_features = extract_lbp_features(images)
    return np.hstack((sift_features, hog_features, lbp_features))

# Predict Function
def predict_image(image_path, model_path, class_names):
    # Load the model
    model = joblib.load(model_path)

    # Load and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    img_processed = preprocess_image(img)

    # Extract features
    features = combine_features([img_processed])

    # Predict
    prediction = model.predict(features)
    return class_names[prediction[0]]  # Map numeric prediction to class name

# Step 5: Make Predictions

# Set paths
image_path = "/content/drive/My Drive/Lung X-Ray Project/Sample_Test/VN/3.jpg"
model_path = "/content/drive/My Drive/Lung X-Ray Project/Models/modelv1.pkl"

# Define class names manually
class_names = ['Lung_Opacity', 'Normal', 'Viral Pneumonia']

# Predict
prediction = predict_image(image_path, model_path, class_names)
print("Predicted Class:", prediction)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 14.5 MB/s eta 0:00:00
Predicted Class: Viral Pneumonia
